In [1]:
from numpy import array
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from numpy import hstack
from keras.layers.merge import concatenate

In [2]:
def split_sequence(sequence, n_steps, n_steps_output):
  X, y = list(), list()
  for i in range(len(sequence)):
   end_ix = i + n_steps
   out_end = end_ix + n_steps_output
   
   if out_end > len(sequence):
     break
     
   seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end]
   X.append(seq_x)
   y.append(seq_y)
  return array(X), array(y)

In [3]:
seq=[10,20,30,40,50,60,70,80,90]

In [4]:
n_steps_input=3
n_steps_output=3

In [5]:
X,y= split_sequence(seq,n_steps_input,n_steps_output)

In [6]:
for i in range(len(X)):
  print(X[i],y[i])

[10 20 30] [40 50 60]
[20 30 40] [50 60 70]
[30 40 50] [60 70 80]
[40 50 60] [70 80 90]


# **Vector Output CNN**

In [ ]:
n_features = 1
X=X.reshape((X.shape[0],X.shape[1],n_features)) 

In [ ]:
X.shape

(4, 3, 1)

In [ ]:
#model
#deninir modelo CNN-1D_multi_step

model= Sequential()
model.add(Conv1D(64,2,activation='relu',input_shape=(n_steps_input,n_features)))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(50,activation='relu'))
model.add(Dense(n_steps_output))
model.compile(optimizer='adam',loss='mse')

In [ ]:
model.fit(X,y,epochs=1500,verbose=True)

Epoch 1/1500
1/1 [==============================] - 0s 2ms/step - loss: 4136.9453
Epoch 2/1500
1/1 [==============================] - 0s 2ms/step - loss: 4002.3086
Epoch 3/1500
1/1 [==============================] - 0s 1ms/step - loss: 3868.9473
Epoch 4/1500
1/1 [==============================] - 0s 2ms/step - loss: 3737.4431
Epoch 5/1500
1/1 [==============================] - 0s 2ms/step - loss: 3608.0923
Epoch 6/1500
1/1 [==============================] - 0s 959us/step - loss: 3480.7065
Epoch 7/1500
1/1 [==============================] - 0s 4ms/step - loss: 3354.2920
Epoch 8/1500
1/1 [==============================] - 0s 2ms/step - loss: 3226.6978
Epoch 9/1500
1/1 [==============================] - 0s 3ms/step - loss: 3100.8696
Epoch 10/1500
1/1 [==============================] - 0s 861us/step - loss: 2979.6895
Epoch 11/1500
1/1 [==============================] - 0s 1ms/step - loss: 2860.7468
Epoch 12/1500
1/1 [==============================] - 0s 1ms/step - loss: 2742.9375
Epoch 13/

In [ ]:
input_test= array([70,80,90])
input_test=input_test.reshape((1,n_steps_input,n_features))


In [ ]:
y_multistep= model.predict(input_test)

In [ ]:
print(y_multistep)

[[100.72048  119.298225 133.20271 ]]


# **Multivariate multi-step **

In [ ]:
# MULTIPLE INPUT SERIES
seq1=array([10,12,30,50,60,70,80,90]) # serie temporal 1 
seq2=array([15,25,35,45,65,75,85,95]) # serie temporal 2

In [ ]:
out_seq= array([seq1[i]+seq2[i] for i in range(len(seq1))])

In [ ]:
def split_sequences_multi(sequence, n_steps, n_steps_output):
  X, y = list(), list()
  for i in range(len(sequence)):
   end_ix = i + n_steps
   out_end = end_ix + n_steps_output-1
   
   if out_end > len(sequence):
     break
     
   seq_x, seq_y = sequence[i:end_ix,:-1], sequence[end_ix-1:out_end,-1]
   X.append(seq_x)
   y.append(seq_y)
  return array(X), array(y)

In [ ]:
# convertir [filas,columnas]
st_1=seq1.reshape((len(seq1),1))
st_2=seq2.reshape((len(seq2),1))
st_3=out_seq.reshape((len(out_seq),1))


In [ ]:
dataset_multi= hstack((st_1,st_2,st_3))
print(dataset_multi)

[[ 10  15  25]
 [ 12  25  37]
 [ 30  35  65]
 [ 50  45  95]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [ ]:
n_steps_input=3
n_steps_output=3

In [ ]:
X_multi,y_multi= split_sequences_multi(dataset_multi,n_steps_input,n_steps_output)

In [ ]:
for i in range(len(X_multi)):
  print(X_multi[i],y_multi[i])

[[10 15]
 [12 25]
 [30 35]] [ 65  95 125]
[[12 25]
 [30 35]
 [50 45]] [ 95 125 145]
[[30 35]
 [50 45]
 [60 65]] [125 145 165]
[[50 45]
 [60 65]
 [70 75]] [145 165 185]


In [ ]:
X_multi.shape


(4, 3, 2)

In [ ]:
y_multi.shape

(4, 3)

In [ ]:
## numer de series - 
n_features= X_multi.shape[2] 

In [ ]:
#MODELO 
modelo_multi_step= Sequential()
modelo_multi_step.add(Conv1D(64,2,activation='relu',input_shape=(n_steps_input,n_features)))
modelo_multi_step.add(MaxPooling1D())
modelo_multi_step.add(Flatten())
modelo_multi_step.add(Dense(50,activation='relu'))
modelo_multi_step.add(Dense(n_steps_output))

In [ ]:
modelo_multi_step.compile(optimizer='adam',loss='mse')
modelo_multi_step.fit(X_multi,y_multi, epochs=1000)

Epoch 1/1000
1/1 [==============================] - 0s 1ms/step - loss: 19142.5156
Epoch 2/1000
1/1 [==============================] - 0s 838us/step - loss: 18769.1621
Epoch 3/1000
1/1 [==============================] - 0s 847us/step - loss: 18395.0977
Epoch 4/1000
1/1 [==============================] - 0s 803us/step - loss: 18032.3516
Epoch 5/1000
1/1 [==============================] - 0s 841us/step - loss: 17675.0332
Epoch 6/1000
1/1 [==============================] - 0s 1ms/step - loss: 17321.5156
Epoch 7/1000
1/1 [==============================] - 0s 2ms/step - loss: 16973.0664
Epoch 8/1000
1/1 [==============================] - 0s 825us/step - loss: 16628.9648
Epoch 9/1000
1/1 [==============================] - 0s 1ms/step - loss: 16288.4746
Epoch 10/1000
1/1 [==============================] - 0s 1ms/step - loss: 15963.2646
Epoch 11/1000
1/1 [==============================] - 0s 2ms/step - loss: 15655.0205
Epoch 12/1000
1/1 [==============================] - 0s 1ms/step - loss: 15

In [ ]:
x_test=array([[70,75],[80,85],[90,95]])
x_test=x_test.reshape((1,n_steps_input,n_features))

In [ ]:
# [185,205,225]
y_test= modelo_multi_step.predict(x_test)
print(y_test)

[[186.6931  204.76012 228.2327 ]]


# **PARALELO MULTISTEP - MULTIVARIADO**

In [ ]:
def split_sequences_multi_parallel(sequence, n_steps, n_steps_output):
  X, y = list(), list()
  for i in range(len(sequence)):
   end_ix = i + n_steps
   out_end = end_ix + n_steps_output 
   
   if out_end > len(sequence):
     break
     
   seq_x, seq_y = sequence[i:end_ix,:], sequence[end_ix:out_end,:]
   X.append(seq_x)
   y.append(seq_y)
  return array(X), array(y)

In [ ]:
# MULTIPLE INPUT SERIES
seq1=array([10,20,30,50,60,70,80,90]) # serie temporal 1 / 
seq2=array([15,25,35,45,65,75,85,95]) # serie temporal 2 / 
seq3=array([2,4,6,8,10,12,14,16])     # serie temporal 3 / 

In [ ]:
# convertir [filas,columnas]
st_1=seq1.reshape((len(seq1),1))
st_2=seq2.reshape((len(seq2),1))
st_3=seq3.reshape((len(seq3),1))

In [ ]:
dataset_parallel=hstack((st_1,st_2,st_3))


In [ ]:
n_steps_input=3
n_steps_output=2

In [ ]:
X_parallel,y_parallel = split_sequences_multi_parallel(dataset_parallel,n_steps_input,n_steps_output)

In [ ]:
for i in range(len(X_parallel)):
  print(X_parallel[i],y_parallel[i])

[[10 15  2]
 [20 25  4]
 [30 35  6]] [[50 45  8]
 [60 65 10]]
[[20 25  4]
 [30 35  6]
 [50 45  8]] [[60 65 10]
 [70 75 12]]
[[30 35  6]
 [50 45  8]
 [60 65 10]] [[70 75 12]
 [80 85 14]]
[[50 45  8]
 [60 65 10]
 [70 75 12]] [[80 85 14]
 [90 95 16]]


In [ ]:
n_features=X_parallel.shape[2]
print(n_features)

3


In [ ]:
salidas= y_parallel.shape[1]*y_parallel.shape[2]
print(salidas)

6


In [ ]:
y_parallel=y_parallel.reshape(y_parallel.shape[0],salidas)


In [ ]:
print(y_parallel.shape)

(4, 6)


In [ ]:
#MODELO 
modelo_multi_parallel= Sequential()
modelo_multi_parallel.add(Conv1D(64,2,activation='relu',input_shape=(n_steps_input,n_features)))
modelo_multi_parallel.add(MaxPooling1D())
modelo_multi_parallel.add(Flatten())
modelo_multi_parallel.add(Dense(50,activation='relu'))
modelo_multi_parallel.add(Dense(salidas))

In [ ]:
modelo_multi_parallel.compile(optimizer='adam',loss='mse')


In [ ]:
modelo_multi_parallel.fit(X_parallel,y_parallel,epochs=1500)

Epoch 1/1500
1/1 [==============================] - 0s 2ms/step - loss: 3452.6218
Epoch 2/1500
1/1 [==============================] - 0s 2ms/step - loss: 3350.4270
Epoch 3/1500
1/1 [==============================] - 0s 1ms/step - loss: 3250.3215
Epoch 4/1500
1/1 [==============================] - 0s 2ms/step - loss: 3152.6929
Epoch 5/1500
1/1 [==============================] - 0s 2ms/step - loss: 3057.1514
Epoch 6/1500
1/1 [==============================] - 0s 2ms/step - loss: 2963.6528
Epoch 7/1500
1/1 [==============================] - 0s 2ms/step - loss: 2872.4653
Epoch 8/1500
1/1 [==============================] - 0s 2ms/step - loss: 2783.5774
Epoch 9/1500
1/1 [==============================] - 0s 1ms/step - loss: 2696.6094
Epoch 10/1500
1/1 [==============================] - 0s 2ms/step - loss: 2610.8296
Epoch 11/1500
1/1 [==============================] - 0s 2ms/step - loss: 2526.0728
Epoch 12/1500
1/1 [==============================] - 0s 1ms/step - loss: 2442.2920
Epoch 13/1500

In [ ]:
x_test=array([[60,65,10],[70,75,12],[80,85,14]])
x_test= x_test.reshape((1,n_steps_input,n_features))

In [ ]:
#[90,95,16] [100,105,18]
paralel_predict= modelo_multi_parallel.predict(x_test)
print(paralel_predict)

[[ 90.435875  98.47114   15.884441 102.010605 107.00273   18.465927]]
